In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdmolops
import networkx as nx
from node2vec import Node2Vec
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os


In [2]:
tox21_file = "data/tox21.csv"
data = pd.read_csv(tox21_file)

smiles_column = "SMILES"  
label_columns = ['SR-HSE','NR-AR', 'SR-ARE', 'NR-Aromatase', 'NR-ER-LBD', 'NR-AhR', 'SR-MMP',\
       'NR-ER', 'NR-PPAR-gamma', 'SR-p53', 'SR-ATAD5', 'NR-AR-LBD'] 

smiles_list = data[smiles_column]
labels = data[label_columns]


labelsDict = {i: label for i,label in enumerate(label_columns)}


In [3]:
datasets = {}

for label in label_columns:
    datasets[label] = data[['SMILES', label]]

SR_HSE = datasets['SR-HSE'].dropna()
NR_AR = datasets['NR-AR'].dropna()
SR_ARE = datasets['SR-ARE'].dropna()
NR_Aromatase = datasets['NR-Aromatase'].dropna()
NR_ER_LBD = datasets['NR-ER-LBD'].dropna()
NR_AhR = datasets['NR-AhR'].dropna()
SR_MMP = datasets['SR-MMP'].dropna()
NR_ER = datasets['NR-ER'].dropna()
NR_PPAR_gamma = datasets['NR-PPAR-gamma'].dropna()
SR_p53 = datasets['SR-p53'].dropna()
SR_ATAD5 = datasets['SR-ATAD5'].dropna()
NR_AR_LBD = datasets['NR-AR-LBD'].dropna()


In [4]:
data

,Formula,FW,DSSTox_CID,SR-HSE,ID,SMILES,Molecule,NR-AR,SR-ARE,NR-Aromatase,NR-ER-LBD,NR-AhR,SR-MMP,NR-ER,NR-PPAR-gamma,SR-p53,SR-ATAD5,NR-AR-LBD
0,C27H25ClN6,468.9806,25848,0.0,NCGC00178831-03,C[n+]1c2cc(N)ccc2cc2ccc(N)cc21.Nc1ccc2cc3ccc(N...,<rdkit.Chem.rdchem.Mol object at 0x0000011999F...,0.0,NaN,1.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN
1,C20H6Br4Na2O5,691.8542,5234,0.0,NCGC00166114-03,O=C([O-])c1ccccc1-c1c2cc(Br)c(=O)c(Br)c-2oc2c(...,<rdkit.Chem.rdchem.Mol object at 0x000001199A4...,0.0,NaN,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,NaN
2,C47H83NO17,934.1584,28909,0.0,NCGC00263563-01,CO[C@@H]1[C@@H](OC)[C@H](C)[C@@](O)(CC(=O)[O-]...,<rdkit.Chem.rdchem.Mol object at 0x000001199A4...,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,1.0,0.0,0.0
3,C52H54N4O12,927.0048,5513,1.0,NCGC00013058-02,CN(C)c1ccc(C(=C2C=CC(=[N+](C)C)C=C2)c2ccccc2)c...,<rdkit.Chem.rdchem.Mol object at 0x000001199A4...,0.0,NaN,NaN,0.0,1.0,1.0,NaN,NaN,NaN,0.0,NaN
4,C66H87N17O14,1342.5025,26683,NaN,NCGC00167516-01,CC(=O)O.CCNC(=O)[C@@H]1CCCN1C(=O)[C@H](CCCNC(=...,<rdkit.Chem.rdchem.Mol object at 0x000001199A4...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8034,C12H22S2,230.4331,27628,0.0,NCGC00254435-01,C1CCC(SSC2CCCCC2)CC1,<rdkit.Chem.rdchem.Mol object at 0x00000119A63...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8035,C7H5Cl2NS,206.0923,21783,0.0,NCGC00255499-01,NC(=S)c1c(Cl)cccc1Cl,<rdkit.Chem.rdchem.Mol object at 0x00000119A63...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8036,CS2,76.1407,3947,0.0,NCGC00258720-01,S=C=S,<rdkit.Chem.rdchem.Mol object at 0x00000119A63...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8037,C3H6N2S,102.1581,601,0.0,NCGC00258846-01,S=C1NCCN1,<rdkit.Chem.rdchem.Mol object at 0x00000119A72...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:

data.shape

(8039, 18)

In [6]:
def smiles_to_graph(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    adjacency_matrix = rdmolops.GetAdjacencyMatrix(mol)
    graph = nx.from_numpy_array(adjacency_matrix)
    for i, atom in enumerate(mol.GetAtoms()):
        graph.nodes[i]['atom_type'] = atom.GetSymbol()
    return graph

graphs = [smiles_to_graph(smiles) for smiles in smiles_list]

valid_data = [(graph, label) for graph, label in zip(graphs, labels.values) if graph is not None]
graphs, labels = zip(*valid_data)
labels = pd.DataFrame(labels) 




[19:27:15] WARNING: not removing hydrogen atom without neighbors


In [7]:
def generate_node_embeddings(graphs, dimensions=64, walk_length=10, num_walks=50):
    embeddings = []
    for graph in tqdm(graphs):
        node2vec = Node2Vec(graph, dimensions=dimensions, walk_length=walk_length, num_walks=num_walks, workers=1, quiet = True)
        model = node2vec.fit()
        node_embeddings = [model.wv[str(node)] for node in graph.nodes]
        molecule_embedding = sum(node_embeddings) / len(node_embeddings)
        embeddings.append(molecule_embedding)
    return embeddings

if not os.path.exists("data/embeddings.pkl"):
    embeddings = generate_node_embeddings(graphs)
else:
    embeddings = pd.read_pickle("data/embeddings.pkl")




100%|██████████| 8039/8039 [03:44<00:00, 35.73it/s]


In [12]:
df = pd.DataFrame(embeddings)
df

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.200776,-0.090112,0.409462,0.018155,-0.008158,-0.156783,-0.101683,0.130622,-0.208105,-0.051638,...,0.378944,0.336896,0.195856,-0.234148,-0.147353,-0.078753,0.146782,-0.207821,0.003552,0.263919
1,0.045015,-0.185515,0.313971,0.091731,0.068513,-0.272349,-0.030110,0.210263,-0.077135,-0.044217,...,0.361024,0.213580,0.224915,-0.260534,-0.076803,-0.085136,0.136665,-0.239424,-0.131847,0.190574
2,0.456437,-0.160811,0.376889,-0.122191,0.128488,-0.479428,0.138289,0.312826,-0.128046,-0.253727,...,0.450283,0.325975,0.151986,-0.192809,-0.188875,0.154892,0.280155,-0.423558,0.082554,0.159891
3,0.433916,-0.263389,0.391413,-0.111709,0.100068,-0.576401,0.251479,0.079591,-0.303162,0.023835,...,0.291984,0.421392,0.215603,-0.185105,-0.253429,0.296994,0.211359,-0.487812,-0.004300,0.221452
4,0.338797,-0.400018,0.410633,0.098293,-0.043371,-0.418567,0.128419,0.134282,-0.374753,-0.097408,...,0.296505,0.328911,0.240457,0.092856,-0.157268,0.548566,-0.055769,-0.298245,-0.009930,-0.201053
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8034,-0.069178,-0.199941,0.244966,0.103731,-0.030214,-0.258177,0.009089,0.016083,-0.137567,-0.125840,...,0.226060,-0.101391,0.031972,-0.255873,-0.057140,0.089747,0.163988,-0.224251,-0.146878,0.125482
8035,-0.059065,-0.188139,0.274162,0.144251,-0.034655,-0.267429,0.071415,0.061799,-0.220369,-0.119223,...,0.234807,-0.096648,0.007730,-0.299132,-0.025982,0.115496,0.098253,-0.247540,-0.180730,-0.021313
8036,-0.002961,0.007460,0.011437,0.007768,-0.009920,-0.008557,0.012323,0.015622,0.000500,-0.004953,...,0.007853,-0.009156,0.006320,-0.010447,0.003487,-0.007912,0.005990,-0.001401,-0.006101,-0.004076
8037,-0.006749,-0.010619,0.041287,0.057916,-0.040422,-0.050518,0.059217,0.094958,-0.018994,0.008303,...,0.061115,-0.044109,0.015838,-0.052808,0.025629,-0.029480,0.013081,-0.028496,-0.067931,-0.017874


In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)

#train_null_mask = np.array(np.logical_not(y_train.isnull().values), dtype=int)
#test_null_mask = np.array(np.logical_not(y_test.isnull().values), dtype=int)

#y_train = y_train.fillna(0)
#y_test = y_test.fillna(0)

#mask_df = pd.DataFrame(null_mask, columns= [col + '_mask' for col in raw_y.columns], index = raw_y.index)

#raw_y = pd.concat([raw_y, mask_df], axis=1)

labels.isna().sum()

In [95]:
"""
null_mask = np.array(np.logical_not(labels.isnull().values), dtype=int)
labels = labels.fillna(0.0)

mask = pd.DataFrame(
    null_mask,
    columns = [str(col) + '_mask' for col in labels.columns],
    index = labels.index
)
labels = pd.concat([labels, mask], axis=1)
"""

In [26]:
X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2, random_state=42)


In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score
from sklearn.neural_network import MLPClassifier

model = XGBClassifier()
#model = MLPClassifier()
model.fit(X_train, y_train.to_numpy().ravel())
y_pred  = model.predict(X_test)

roc_auc_score(y_test, y_pred, average='micro')


In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cbar = False)

In [ ]:
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier

models = {
    'RandomForest' : RandomForestClassifier(n_jobs=-1),
    'SVC' : SVC(),
    'XGBoost' : XGBClassifier(),
    'LightGBM' : LGBMClassifier(),
    'LR' : LogisticRegression(),
    'KNN' : KNeighborsClassifier(12)
}

print('Generated models')
pre = []
roc_auc = []
for m in tqdm(models.keys()):
    print('Training model: ', m)
    clf = MultiOutputClassifier(models[m])
    clf.fit(X_train, y_train)
    print("{m} trained, predicting and scoring")
    y_pred = clf.predict(X_test)
    pre.append(accuracy_score(y_test, y_pred))
    roc_auc.append(roc_auc_score(y_test, y_pred, average='weighted'))




# Plot histogram of f1 and roc auc scores of each model
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot f1 scores
sns.barplot(x=list(models.keys()), y=pre, ax=axes[0])
axes[0].set_title('Precision score of Models')
axes[0].set_xlabel('Models')
axes[0].set_ylabel('F1 Score')

# Plot ROC AUC scores
sns.barplot(x=list(models.keys()), y=roc_auc, ax=axes[1])
axes[1].set_title('ROC AUC Scores of Models')
axes[1].set_xlabel('Models')
axes[1].set_ylabel('ROC AUC Score')

plt.tight_layout()
plt.show()


In [ ]:
#clf = RandomForestClassifier(class_weight='balanced', random_state=42)
#clf = XGBClassifier()
clf = SVC(
    #class_weight='balanced',
    kernel='sigmoid'
)
model = MultiOutputClassifier(clf)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns=y_train.columns.tolist())
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(classification_report(y_test, y_pred, target_names=y_train.columns))

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

mat = multilabel_confusion_matrix(y_test, y_pred)
mat
fig, axes = plt.subplots(3, 4, figsize=(15, 10))
axes = axes.ravel()

for i, (ax, label) in enumerate(zip(axes, label_columns)):
    sns.heatmap(mat[i], annot=True, fmt='d', ax=ax, cmap='coolwarm', cbar=False)
    ax.set_title(label)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('True')

plt.tight_layout()
plt.show()

In [ ]:
len(X_train)